In [1]:
!pip install gradio --quiet

import gradio as gr
import time

print(gr.__version__)


     |████████████████████████████████| 2.2 MB 11.6 MB/s 
     |████████████████████████████████| 1.9 MB 54.3 MB/s 
     |████████████████████████████████| 206 kB 59.6 MB/s 
     |████████████████████████████████| 63 kB 1.1 MB/s 
     |████████████████████████████████| 961 kB 58.6 MB/s 
     |████████████████████████████████| 3.2 MB 60.6 MB/s 
2.2.8


In [2]:
!pip install transformers --quiet
!pip install allennlp==1.0.0 allennlp-models==1.0.0 --quiet
#!pip install --user gradio

from urllib import request
from bs4 import BeautifulSoup as bs
import re
import nltk
import heapq
from transformers import pipeline
#import gradio as gr

nltk.download('punkt')
nltk.download('stopwords')

     |████████████████████████████████| 2.6 MB 13.4 MB/s 
     |████████████████████████████████| 895 kB 61.3 MB/s 
     |████████████████████████████████| 636 kB 72.8 MB/s 
     |████████████████████████████████| 3.3 MB 65.5 MB/s 
     |████████████████████████████████| 473 kB 14.3 MB/s 
     |████████████████████████████████| 282 kB 86.7 MB/s 
     |████████████████████████████████| 124 kB 87.2 MB/s 
     |████████████████████████████████| 753.2 MB 12 kB/s 
     |████████████████████████████████| 259 kB 85.1 MB/s 
     |████████████████████████████████| 674 kB 55.2 MB/s 
     |████████████████████████████████| 131 kB 61.5 MB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 1.2 MB 63.5 MB/s 
     |████████████████████████████████| 5.6 MB 60.9 MB/s 
     |████████████████████████████████| 7.8 MB 40.8 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 127 kB 89.7 MB/s 
ERROR: pip's depen

True

In [3]:
from allennlp.predictors.predictor import Predictor

summarizer = pipeline("summarization") # model="t5-small"
# predictor = Predictor.from_path("https://s3-us-west-2.amazonaws.com/allennlp/models/bidaf-model-2017.09.15-charpad.tar.gz")
predictor = Predictor.from_path("https://storage.googleapis.com/allennlp-public-models/bidaf-model-2020.03.19.tar.gz")


100%|██████████| 45959361/45959361 [00:01<00:00, 38143449.00B/s]


In [6]:

#First compile everything into a function
def main(website, question, bullet_points):

    url = website
    TotalContent = ""

    htmlDoc = request.urlopen(url)

    soupObj = bs(htmlDoc, 'html.parser')
    paragraphContents = soupObj.findAll('p')

    for content in paragraphContents:

        TotalContent += content.text



    total_cleaned_content = re.sub(r'\[[0-9]*\]', ' ', TotalContent)
    total_cleaned_content = re.sub(r'\s+', ' ', total_cleaned_content)

    sentence_tokens = nltk.sent_tokenize(total_cleaned_content)

    total_cleaned_content = re.sub(r'[^a-zA-Z]', ' ', total_cleaned_content)
    total_cleaned_content = re.sub(r'\s+', ' ', total_cleaned_content)

    #print(sentence_tokens)

    words_tokens = nltk.word_tokenize(total_cleaned_content)

    stopwords = nltk.corpus.stopwords.words('english')

    word_frequencies = {}

    for word in words_tokens:

        if word not in stopwords:

            if word not in word_frequencies.keys():

                word_frequencies[word] = 1
            else:

                word_frequencies[word] += 1


    #print(words_tokens)

    maximum_frequency = max(word_frequencies.values())

    for word in word_frequencies.keys():
        
        word_frequencies[word] = (word_frequencies[word]/maximum_frequency)

    #print(word_frequencies)

    sentence_scores = {}

    for sentence in sentence_tokens:

        for word in nltk.word_tokenize(sentence.lower()):

            if word in word_frequencies.keys():

                if (len(sentence.split(' '))) < 30:

                    if sentence not in sentence_scores.keys():
                        sentence_scores[sentence] = word_frequencies[word]
                    else:
                        sentence_scores[sentence] += word_frequencies[word]


    points = heapq.nlargest(20, sentence_scores, key=sentence_scores.get)

    total_text = ""

    for i in points:

        total_text += i

    string = """"""

    for i in range(bullet_points):
        
        string += "\n - " + points[i]

    

    summary = summarizer(total_text, max_length=128, min_length=64, do_sample=False)

    result=predictor.predict(
    passage=TotalContent,
    question=question
    )

    return summary[0]["summary_text"], string, result['best_span_str']



In [ ]:
gr.Interface(
  fn=main, 
  inputs=["text", "text", gr.inputs.Slider(0, 20, step=1)],
  outputs=["text", "text", "text"]).launch(debug=True)